In [ ]:
# A basic latitude/longitude webscraper for postal codes in Singapore
# It combines OneMapSG API with Google Places API to scrape more dated postal codes while minimise costs of using Google's services

# fill in Google API key
google_api_key = " "

# [SETUP] 
import requests
import json
import csv
import pandas as pd

geocoded = []
missing = []  
# Note: OneMap and GoogleMaps historical records extend to a certain date. 
# OpenStreetMap is a good third source for more historical postal codes, but is not included in this script.

# [SETUP] input pathname; read csv file with single column of postal codes and no header
pd.read_csv(str(path_to_file))
search = df_pcode[[0]].values.tolist()

# [SCRAPING] OneMap API
for pcode in search:
    time.sleep(0.3)  # being courteous by sleep the scraping to not overload the server    
    search_val == pcode.zfill(6)
    
    url = 'https://developers.onemap.sg/commonapi/search?searchVal=' + search_val + '&returnGeom=Y&getAddrDetails=Y'
    try:
        request = (requests.get(url))
        data = json.loads(request.text)
        datalist=data["results"]
        
        try:
            postalcode = datalist[0]['POSTAL']
            try:
                postalcode = datalist[0]['POSTAL']
                if postalcode == value:
                    latitude = datalist[0]['LATITUDE']
                    longitude = datalist[0]['LONGITUDE']
                    to_add = [value, latitude, longitude]
                    print(to_add)
                    geocoded.append(to_add)
                else:
                    print(search_val, ' not found. Added to Missing list')
                    missing.append(search_val)
            except:
                print(search_val, ' not found. Added to Missing list')
                missing.append(search_val)
        except:
            print(search_val, ' not found. Added to Missing list')
            missing.append(search_val)
            
    # [ERROR HANDLING] Use GoogleMaps when OneMap does not have postal code
    # need to obtain an API key from Google.
    except json.decoder.JSONDecodeError:
        print('trying google')
        gurl = 'https://maps.googleapis.com/maps/api/place/textsearch/json?query=Singapore+' + search_val + '&key=' + google_api_key
        grequest = (requests.get(gurl))
        gdata = json.loads(grequest.text)
        gdatalist = gdata["results"]
        try:
             if value in gdatalist[0]["formatted_address"]:
                lat=gdatalist[0]['geometry']['location']['lat']
                long=gdatalist[0]['geometry']['location']['lng']
                glat=str(lat)
                glong=str(long)
                to_add = [value, lat, long]
                geocoded.append(to_add)
                    
        # [ERROR HANDLING] when GoogleMaps does not have postal code
        except IndexError:
            print(value + ' IndexError')
            missing.append(value)
        except:
            print(value + ' error')
            missing.append(value)
        
# [SAVE] save Geocoded list
print('\n Saving files')
geocoded_addr = pd.DataFrame(geocoded) # using pandas for large dataset for efficiency 
geocoded_addr.to_csv("geocoded.csv", header=["Postal Code", "Latitude", "Longitude"], index=False)
print('Geocoded postal codes saved')

# [SAVE] save Missing list
with open ('geocode_missing.csv', 'w') as file:
    writer=csv.writer(file, delimiter='\t', lineterminator='\n',)
    writer.writerow(missing)
print('Missing postal codes saved')
                
# [Complete!]                
end = time.time()  #calculate total time taken
print('\nGeocoding complete!')